In [ ]:
from HTML import config as Config
from HTML.dataset import *
from HTML.config import ratio, nominal, ordinal, meaningless
# from HTML.preprocessing import preprocessing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df_train = pd.read_csv(Config.train_path)
df_train.head()

In [ ]:
df_train.shape

In [ ]:
def preprocessing(df):
    result = df.copy()
    result = result.drop(columns=['company', 'ID', 'index'], errors='ignore')
    result = result.fillna({
        'children': 0,
        'country': 'None',
        'agent': 'None'
    })
    result['stays_in_nights'] = result['stays_in_weekend_nights'] + result['stays_in_week_nights']
    result['same_room_type'] = (df['assigned_room_type'] == df['reserved_room_type']).astype(str)
    categories = sorted(list(set(result.columns) & set(ordinal+nominal+meaningless+['same_room_type'])))
    numeral = sorted(list(set(result.columns) - set(categories)))
    result[categories] = result[categories].astype(str)
    result[numeral] = result[numeral].astype(float)
    return result

In [ ]:
df_train = preprocessing(df_train)
df_train.head()

In [ ]:
df_train.shape

In [ ]:
y_train_is_canceled = df_train['is_canceled'].to_numpy()
y_train_is_canceled_adr = df_train['adr'].to_numpy()
y_train_adr = df_train[(df_train['adr'] < 5000) & (df_train['is_canceled'] == '0')]['adr'].to_numpy()

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
enc = OneHotEncoder(drop='if_binary', sparse=False)

In [ ]:
not_features = {'arrival_date_year', 'adr', 'reservation_status', 'reservation_status_date', 'is_canceled'}

In [ ]:
categories_train = df_train[sorted(list(set(df_train.columns[df_train.dtypes == object]) - not_features))].astype(str)
cat = enc.fit_transform(categories_train.to_numpy())

In [ ]:
categories_train.shape

In [ ]:
cat.shape

In [ ]:
number = df_train[sorted(list(set(df_train.columns[df_train.dtypes == float]) - not_features))]
num = number.to_numpy()

In [ ]:
num.shape

In [ ]:
tmp = np.concatenate((cat, num), axis=1)
x_train_is_canceled = tmp.copy()
x_train_adr = tmp[(df_train['adr'] < 5000) & (df_train['is_canceled'] == '0'),:].copy()

In [ ]:
# dummies = pd.get_dummies(
#         data=df_train[set(df_train.columns) - {'reservation_status', 'reservation_status_date', 'is_canceled'}],
#         columns=set(nominal+ordinal) - {'reservation_status', 'reservation_status_date', 'is_canceled'},
#         drop_first=True,
#         dummy_na=True)
# dummies = dummies[sorted(set(dummies.columns) - {'index', 'ID'} - set(meaningless))]
# x_train_res = dummies[sorted(list(set(dummies.columns) - {'adr'}))].to_numpy()
# x_train_adr = dummies[dummies['adr'] < 5000][sorted(list(set(dummies.columns) - {'adr'}))].to_numpy()

In [ ]:
print('adr:', x_train_adr.shape, y_train_adr.shape)
print('res:', x_train_is_canceled.shape, y_train_is_canceled.shape)

In [ ]:
from xgboost import XGBClassifier, XGBRegressor

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

params = {
    'n_estimators': [100],
    'learning_rate': [.3],
    'max_depth': [3,6,8],
    'min_child_weight': [0,1,3],
    'colsample_bytree': [.3,.5],
    'subsample': [.3,.5],
    'gamma': [1,10,20],
    'objective': ['reg:squarederror', 'reg:pseudohubererror'],
    'n_jobs': [-1],
    'random_state': [1126],
    'tree_method': ['gpu_hist'],
    'predictor': ['gpu_predictor'],
    'deterministic_histogram': [False]
}

regr_cv = RandomizedSearchCV(
    XGBRegressor(),
    params,
    scoring=['neg_mean_absolute_error', 'neg_mean_squared_error'],
    refit='neg_mean_squared_error',
    n_iter=50,
    n_jobs=3,
#     pre_dispatch=12,
    verbose=10,
    return_train_score=True
)

regr_cv.fit(x_train_adr, y_train_adr)

In [ ]:
# result = pd.DataFrame(regr_cv.cv_results_)
# result.head()

In [ ]:
regr_cv.best_estimator_.get_params()

In [ ]:
regr = regr_cv.best_estimator_
# regr = XGBRegressor(
#     objective='reg:squarederror',
#     n_estimators=100,
#     learning_rate=.3,
#     max_depth=6,
#     min_child_weight=1,
#     colsample_bytree=.5,
#     subsample=.5,
#     gamma=1,
#     n_jobs=-1,
#     random_state=1126,
#     tree_method='gpu_hist',
#     predictor='gpu_predictor',
#     deterministic_histogram=False
# )

In [ ]:
regr.fit(x_train_adr, y_train_adr)

In [ ]:
np.argsort(regr.feature_importances_)[::-1][:10]

In [ ]:
from xgboost import plot_importance
plot_importance(regr, importance_type='gain', max_num_features=10, grid=False)
plt.show()

In [ ]:
y_train_is_canceled

In [ ]:
# from sklearn.model_selection import RandomizedSearchCV

# params = {
#     'n_estimators': [100],
#     'learning_rate': [.3],
#     'max_depth': [3,6,8],
#     'min_child_weight': [0,1,3],
#     'colsample_bytree': [.3,.5],
#     'subsample': [.3,.5],
#     'gamma': [1,10,20],
#     'objective': ['binary:logistic', 'binary:logitraw', 'count:poisson', 'reg:tweedie'],
#     'n_jobs': [-1],
#     'random_state': [1126],
#     'tree_method': ['gpu_hist'],
#     'predictor': ['gpu_predictor'],
#     'deterministic_histogram': [False]
# }

# clf_cv = RandomizedSearchCV(
#     XGBClassifier(),
#     params,
#     scoring=['accuracy', 'f1'],
#     refit='f1',
#     n_iter=100,
#     n_jobs=2,
# #     pre_dispatch=12,
#     verbose=10,
#     return_train_score=True
# )

# clf_cv.fit(x_train_is_canceled, y_train_is_canceled.astype(int))

In [ ]:
# clf_cv.best_estimator_.get_params()

In [ ]:
# clf = clf_cv.best_estimator_
clf = XGBClassifier(
    objective='count:poisson',
    n_estimators=100,
    learning_rate=.3,
    max_depth=6,
    subsample=.7,
    colsample_bytree=.7,
    gamma=1,
    max_delta_step=0.699999988,
    n_jobs=-1,
    random_state=1126,
    tree_method='gpu_hist',
    predictor='gpu_predictor',
    deterministic_histogram=False
)
clf.fit(x_train_is_canceled, y_train_is_canceled)

In [ ]:
np.argsort(clf.feature_importances_)[::-1][:10]

In [ ]:
plot_importance(clf, importance_type='gain', max_num_features=10, grid=False)
plt.show()

In [ ]:
number.columns

In [ ]:
categories_train.columns

In [ ]:
c_unique = [len(categories_train[c].unique()) for c in categories_train.columns]
sum(c_unique)

In [ ]:
np.cumsum(c_unique)

In [ ]:
plt.plot(np.cumsum(c_unique))

In [ ]:
pred_adr = regr.predict(x_train_is_canceled)
prob_is_canceled = clf.predict_proba(x_train_is_canceled)

In [ ]:
np.mean(np.abs(pred_adr - y_train_is_canceled_adr))

In [ ]:
np.mean((pred_adr - y_train_is_canceled_adr)**2)

In [ ]:
pred_is_canceled = 1- prob_is_canceled[:,0].copy()

In [ ]:
pred_is_canceled

In [ ]:
tmp = clf.predict(x_train_is_canceled)

In [ ]:
tmp

In [ ]:
tmp = tmp.astype(int)
np.unique(tmp)

In [ ]:
tmp

In [ ]:
1 - np.mean(np.abs(tmp - y_train_is_canceled.astype(int)))

In [ ]:
new_df_train = df_train.copy().reset_index()
new_df_train['revenue'] = new_df_train['adr'] * np.sum(new_df_train[['stays_in_weekend_nights', 'stays_in_week_nights']], axis=1) * (1-new_df_train['is_canceled'].astype(int))
new_df_train['pred_adr'] = pred_adr
# new_df_train['pred_is_canceled'] = tmp
new_df_train['pred_is_canceled'] = pred_is_canceled
new_df_train['pred_revenue'] = new_df_train['pred_adr'] * np.sum(new_df_train[['stays_in_weekend_nights', 'stays_in_week_nights']], axis=1) * (1-new_df_train['pred_is_canceled'])
new_df_train[['revenue', 'pred_revenue']].head()

In [ ]:
plt.scatter(new_df_train['pred_revenue'], new_df_train['revenue'], alpha=.3)
plt.show()

In [ ]:
np.mean((new_df_train['revenue'] - new_df_train['pred_revenue']).to_numpy() ** 2)

In [ ]:
np.mean(np.abs((new_df_train['revenue'] - new_df_train['pred_revenue']).to_numpy()))

In [ ]:
dates = ['arrival_date_year', 'arrival_date_day_of_month']
new_df_train[dates] = new_df_train[dates].astype(int)
new_df_train = add_arrival_date(new_df_train)

In [ ]:
new_df_train_sum = new_df_train[['revenue', 'pred_revenue', 'arrival_date']].groupby('arrival_date').sum()

In [ ]:
new_df_train_sum.head()

In [ ]:
plt.scatter(new_df_train_sum['pred_revenue'], new_df_train_sum['revenue'], alpha=.3)
plt.show()

In [ ]:
np.mean((new_df_train_sum['revenue'] - new_df_train_sum['pred_revenue']).to_numpy() ** 2)

In [ ]:
np.mean(np.abs((new_df_train_sum['revenue'] - new_df_train_sum['pred_revenue']).to_numpy()))

In [ ]:
y_train = pd.read_csv(Config.train_label_path)
train = new_df_train_sum.merge(y_train, left_index=True, right_on='arrival_date')

In [ ]:
plt.scatter(train['pred_revenue'], train['label'], alpha=.5)
plt.scatter(train['revenue'], train['label'], alpha=.5)
plt.show()

In [ ]:
def get_pred(revenue, threshold):
    pred = np.empty(len(revenue))
    label   = revenue // 1e4
    residual = revenue % 1e4
    for i, (l, res) in enumerate(zip(label, residual)):
        if np.abs(res - 5000) <= (5000 - threshold):
            pred[i] = l
        else:
            pred[i] = min(9, max(0, l + res / 1e4 - int(res < 5000) + .5 + res/10000))
    return pred

In [ ]:
thresholds = np.arange(0, 5000, 100)
best_e = np.inf
best_t = -1
for i, t in enumerate(thresholds):
    pred = get_pred(train['pred_revenue'].copy(), t)
    e = np.mean(np.abs(pred - train['label']))
    if e < best_e:
        best_e = e
        best_t = t
    print(f'\r[{i+1}/{len(thresholds)}] threshold: {t} / MAE: {e}', end='')
print(f'\nbest threshold: {best_t} / best MAE: {best_e}')

In [ ]:
np.mean(np.abs(np.floor(train['revenue'] / 10000) - train['label']))

In [ ]:
np.mean(np.abs(np.floor(train['pred_revenue'] / 10000) - train['label']))

In [ ]:
np.mean(np.abs(train['pred_revenue'] / 10000 - train['label']))

In [ ]:
plt.plot(train['revenue'])
plt.show()
plt.plot(train['pred_revenue'])
plt.show()

In [ ]:
df_test = pd.read_csv(Config.test_path)
df_test.head()

In [ ]:
df_test = preprocessing(df_test)
df_test.head()

In [ ]:
num.shape

In [ ]:
df_test.shape

In [ ]:
categories_test = df_test[sorted(list(set(df_test.columns[df_test.dtypes == object]) - not_features))].astype(str)
for c in categories_train.columns:
    if 'None' in categories_train[c].unique():
        unkown = set(categories_test[c].unique()) - set(categories_train[c].unique())
        categories_test[c][categories_test[c].isin(unkown)] = 'None'
cat = enc.transform(categories_test.to_numpy())

In [ ]:
categories_test.shape

In [ ]:
cat.shape

In [ ]:
number = df_test[sorted(list(set(df_test.columns[df_test.dtypes == float]) - not_features))]
num = number.to_numpy()

In [ ]:
num.shape

In [ ]:
tmp = np.concatenate((cat, num), axis=1)
x_test = tmp.copy()

In [ ]:
print('test:', x_test.shape)

In [ ]:
pred_adr = regr.predict(x_test)
prob_is_canceled = clf.predict_proba(x_test)

In [ ]:
pred_is_canceled = 1- prob_is_canceled[:,0].copy()

In [ ]:
pred_is_canceled

In [ ]:
tmp = clf.predict(x_test)
tmp = tmp.astype(int)
np.unique(tmp)

In [ ]:
tmp

In [ ]:
new_df_test = df_test.copy().reset_index()
new_df_test['pred_adr'] = pred_adr
# new_df_test['pred_is_canceled'] = tmp
new_df_test['pred_is_canceled'] = pred_is_canceled
new_df_test['pred_revenue'] = new_df_test['pred_adr'] * np.sum(new_df_test[['stays_in_weekend_nights', 'stays_in_week_nights']], axis=1) * (1-new_df_test['pred_is_canceled'])
new_df_test[['pred_adr', 'pred_is_canceled', 'pred_revenue']].head()

In [ ]:
new_df_train['is_canceled'] = new_df_train['is_canceled'].astype(int)
new_df_train[['pred_adr', 'adr', 'pred_is_canceled', 'is_canceled', 'pred_revenue', 'revenue']].describe()

In [ ]:
new_df_test[['pred_adr', 'pred_is_canceled', 'pred_revenue']].describe()

In [ ]:
dates = ['arrival_date_year', 'arrival_date_day_of_month']
new_df_test[dates] = new_df_test[dates].astype(int)
new_df_test = add_arrival_date(new_df_test)

In [ ]:
new_df_test_sum = new_df_test.groupby('arrival_date').sum()

In [ ]:
new_df_test_sum.head()

In [ ]:
new_df_test_sum['pred_revenue'].describe()

In [ ]:
new_df_train_sum['revenue'].hist(density=True, bins=range(0,100000, 10000), alpha=.4)
new_df_train_sum['pred_revenue'].hist(density=True, bins=range(0,100000, 10000), alpha=.4)
new_df_test_sum['pred_revenue'].hist(density=True, bins=range(0,100000, 10000), alpha=.4)
plt.legend(['train (actual)', 'train (pred)', 'test'])
plt.show()

In [ ]:
plt.plot(new_df_test_sum['pred_revenue'])
plt.show()

In [ ]:
len(new_df_train_sum)

In [ ]:
len(new_df_test_sum)

In [ ]:
plt.plot((new_df_train_sum['revenue'].to_list() + new_df_test_sum['pred_revenue'].to_list())[-200:])
plt.show()

In [ ]:
test_index = new_df_test_sum.index

In [ ]:
label = np.floor(new_df_test_sum['pred_revenue'].to_numpy() / 10000)

In [ ]:
plt.hist(train['label'], density=True, alpha=.7)
plt.hist(label, bins=range(0,10), density=True, alpha=.7)
plt.show()

In [ ]:
from HTML.save import save_prediction
save_prediction('../outputs//xgboost2_is_canceled_cv_enc_only_non_canceled.csv', test_index, label)